In [ ]:
import speech_recognition as sr
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sp
from matplotlib import pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pyttsx3
from try_zoro import finalzoro
from itertools import zip_longest
import csv
import requests

from bs4 import BeautifulSoup as bs

import pywhatkit
import datetime
import wikipedia
import pyjokes
from subprocess import call
import sys  
import subprocess as sp
import operator
import webbrowser
import os
from wonderwords import RandomSentence
# Initialize the speech recognition, text-to-speech, and microphone objects
listener = sr.Recognizer()
my_mic = sr.Microphone(device_index=0)
speaker=pyttsx3.init()
speaker.setProperty('rate',150)
# List of available menu options
menu= ["play a song ","time","who is (wikipedia)","what is (calculator)","joke","write a note","open a note","open youtube","open google","shutdown","stop"]
# list1=['1','2','3','4','5','6','7','8','9']
#detect wake word
def listen_for_wake_word(wake_word="zoro"):
    try:
        with my_mic as source:
            print(f"Wake up {wake_word}")
            listener.adjust_for_ambient_noise(source)
            
            while True:
                voice = listener.listen(source)
                command = listener.recognize_google(voice, language='en',show_all=False)
                command = command.lower()
                
                if wake_word in command:
                    print("How can i help you?")
                    speaker.say("How can i help you?")
                    speaker.runAndWait()
                    return True
                
    except:
        print("Error, try again")
    return False
def main():
    wake_word = "zoro"
    
    while True:
        wake_word_detected = listen_for_wake_word(wake_word)
        
        if wake_word_detected:
            run_zoro()
        break     

#assistant commands
def take_command(wake_word="zoro"):
    command = ""
    for i in range(3):
        try:
            with my_mic as source:
                print(f"{wake_word} Listening...")
                listener.adjust_for_ambient_noise(source)
                voice = listener.listen(source)
                command = listener.recognize_google(voice, language='en')
                command = command.lower()
                
                if command.startswith(wake_word):
                    command = command.replace(wake_word, "").strip()
                    print(command)
                
        except:
            speaker.say("say it again")
            speaker.runAndWait()
        return command


#assistant commands
def run_zoro():
    command = take_command()
    print(command)
    try:
        #stop the run
        if 'stop' in command:
            speaker.say("zoro is outta here")
            speaker.runAndWait()
            sys.exit()
        #commands menu
        elif 'menu' in command:
            speaker.say("I can do alot of things here is a menu you can choose from")
            speaker.runAndWait()
            numbered_points = "/n".join([f"{i+1}. {point}" for i, point in enumerate(menu)])
            print(numbered_points)
        #know the time
        elif 'time' in command:
            time = datetime.datetime.now().strftime('%I:%M %p')
            speaker.say('Current time is ' + time)
            speaker.runAndWait()
        #wikipedia
        elif 'who is' in command:
            person = command.replace('who is', '')
            info = wikipedia.summary(person, 1)
            print(info)
            speaker.say(info)
            speaker.runAndWait()
        
        elif  'collect ' in command:
            search = command.replace('dataset', '')
            search1 = command.replace('data set', '')
            data = wikipedia.search(search, 1)
            file_path = "Dataset.txt"  
            Moz3g(file_path,data)

        elif 'test ' in command:
            speaker.say("test done successfully")
            speaker.runAndWait()
            zoro_scraping()    
            
        #make a joke
        elif 'joke' in command:
            speaker.say(pyjokes.get_joke())
            speaker.runAndWait()
        #play a video from youtube 
        elif 'play' in command:
            song = command.replace('play', '')
            speaker.say('playing ' + song)
            speaker.runAndWait()
            pywhatkit.playonyt(song) 

        # Write the note content to the file
        elif 'write note' in command:
            speaker.say('note openned')
            speaker.runAndWait()
            create_notepad()
        #to open saved note   
        elif 'open note' in command:
            os.startfile("adham.txt" )
        #calculator
        elif 'what is' in command:
            command = command.replace('what is', '')
            x=eval_binary_expr(*(command.split()))
            print(x)
            speaker.say(x)
            speaker.runAndWait()
            
        # elif 'start' in command:
        #     speaker.say('''Dr.Esraa Afify Lecturer at Eru,Graduated from Helwan University,have many skills in python,machine learning,deep learning,data analysis and much more, 
        #                 today it is discussion day lets start with menu''')
        #     speaker.runAndWait()

        # elif 'generate' in command:
        #     x = command.replace('generate', '')
        #     y=convert_word_to_numeral(x)
        #     if type(y) == int:
        #         for i in range(y):
        #             create_notepad(generate_sentence())
        #             speaker.say(f'i successfully generated {y} sentence')
        #     elif type(y)==str :
        #         speaker.say("that is not a number")
        #         print("invalid zeft")


        #command to open youtube
        elif 'open youtube' in command:
            webbrowser.open("www.youtube.com")
            web = command.replace('open youtube', '')
            speaker.say('openning youtube ' + web)
            speaker.runAndWait()
        #command to open google
        elif 'open google' in command:
            webbrowser.open("www.google.com")
            web = command.replace('open google', '')
            speaker.say('openning google ' + web)
            speaker.runAndWait()
        #shutdown the system
        elif 'shut down' in command:
            speaker.say('Is this the end after all this suffering then good bye and yes you out')
            speaker.runAndWait()
            os.system("shutdown /s /t 1")
        elif 'record' in command:
            speaker.say("starting to record")
            speaker.runAndWait()
            create_notepad()
        elif "my project" in command:
            speaker.say("give me second")
            speaker.runAndWait()
            NLP1()
    except sr.UnknownValueError:
        speaker.say('Please say the command again.')
        speaker.runAndWait()
 
#operator selection
def get_operator_fn(op):
    return {
        '+' : operator.add,
        '-' : operator.sub,
        'x' : operator.mul,
        '*' : operator.mul,
        'divided' :operator.__truediv__,
        '/' :operator.__truediv__,
        'mod' : operator.mod,
        '^' : operator.xor,
        }[op]

def eval_binary_expr(op1, oper, op2):
    op1,op2 = int(op1), int(op2)
    return get_operator_fn(oper)(op1, op2)
# Write the note content to the file
def create_notepad():
    file_path = "NLP.txt"  
   
    with my_mic as source:
            speaker.say("what do you want to write?")
            listener.adjust_for_ambient_noise(source)
            voice = listener.listen(source)
            text = listener.recognize_google(voice, language='en')
            text = text.lower()
    
    
    with open(file_path, "w") as file:
        file.write(text + "/n")
    
    speaker.say("Note created.")


def Moz3g(file_path,data):
    with open(file_path, "a") as file:
        file.write(data + "/n")


def zoro_scraping():
    result=requests.get("https://play.google.com/store/apps/details?id=com.egypt.hardeesburger.fastfood&hl=en&gl=US&pli=1")
    source=result.content
    soup=bs(source,"lxml")
    restuarant_review=soup.find_all("div",{"class":"h3YV2d"})
    restuarant_review1=[]
    for i in range(len(restuarant_review)):
        restuarant_review1.append(restuarant_review[i].text)
    file_list=[restuarant_review1]   
    exported=zip_longest(*file_list)
    with open("yaraaaaaaab.csv","a") as Jobs:
        wr=csv.writer(Jobs)
        wr.writerow(["Reviews"])
        wr.writerows(exported)
        print("scraped done")

def NLP1():
    call(["python","D:/JOU/NLP/newNLP_ALL.ipynb"])
    
    speaker.say("i printed your  accuracy score ")
    speaker.runAndWait()
    print(finalzoro)

while True:
    main()